# AMGeO CEDAR Workshop 2021

![AMGeO Logo](./static/amgeoLogo.svg)

Welcome to AMGeO's workshop for CEDAR 2021. In this workshop, we hope to 

- Give a brief introduction to AMGeO

- Get you registered with AMGeO and its data providers

- Introduce our new API for interacting with AMGeO entirely with jupyter notebooks

- Have you try out our new API for yourself!

## AMGeO Introduction

AMGeO is a collaborative data science platform for the geospace science community for bringing together a diverse set of heterogeneous geospace observations from NSF-funded facility programs and individual community users to obtain complete maps of high-latitude ionospheric electrodynamics for scientific discovery and space weather research. The platform is made of the AMGeO open-source software and web application services that facilitate the data acquisition and pre-processing steps that are otherwise prohibitively labor-intensive. It is developed at the University of Colorado Boulder by the AMGeO Team, with support from the NSF Earth Cube program.

For more information, please see our website [here](https://amgeo.colorado.edu/home)

**We would like to thank all of our data providers:**

[SuperMAG](https://supermag.jhuapl.edu/)

[SuperDARN](http://vt.superdarn.org/tiki-index.php)

[AMPERE](http://ampere.jhuapl.edu/)

## Registration for AMGeO

![AMGeO Pipeline](./static/AMGeOPipeline.png)

To use AMGeO, users will need to register on 3 different platforms:

- AMGeO (https://amgeo.colorado.edu/register)

- SuperMag (https://supermag.jhuapl.edu/mag/)

- AMPERE (http://ampere.jhuapl.edu/)

For more information on why you need to register, please see https://amgeo.colorado.edu/about#data-policy

## New AMGeO API Demo

### Setting up your AMGeO Environment

Now that we are familiar with what AMGeO does and are registered, let's try out the new API! First step is to import AMGeO for this notebook.

<div class="alert alert-block alert-danger">
    <b>Warning:</b> If you have not registered for AMGeO, you will not be able to import the API. <br><br>
    Please follow the above links in 'Registration for AMGeO' to complete this step
</div>

In [ ]:
# Import AMGeOApi
from AMGeO.api import AMGeOApi

Next, create a new API class using the AMGeOApi contructor. This will return an AMGeOApi class for us to utilize

```python
# creates a new AMGeOApi class
AMGeOApi()
```

In [ ]:
api = AMGeOApi()

This will allow us to access helpful information, such as the current output directory of where AMGeO data will be generated, or changing your registration information.

First, see where AMGeO's current output directory is by running the method ```get_output_dir```

```python
# returns the current output directory for AMGeO
AMGeOApi.get_output_dir()

# Example
api = AMGeOApi()
api.get_output_dir()
>>> '~/amgeo_output_dir'
```

In [ ]:
api.get_output_dir()

This output directory is the default that AMGeO will set for any user. If you look at the directory we are currently in, we already have some data generated by AMGeO for you to use. To start using that data, we must set our output directory to this directory using ```set_output_dir```

```python
# sets the current output directory for AMGeO
AMGeOApi.set_output_dir('/the/directory')

# Example
api = AMGeOApi()
api.set_output_dir('./my/local/directory')
api.get_output_dir()
>>> './my/local/directory'
```

In [ ]:
api.set_output_dir('./api_demo_output')

Let's verify that this worked by testing that running ```get_output_dir``` now is pointing to the correct location

In [ ]:
assert(api.get_output_dir() == './api_demo_output')

### Generating AMGeO Maps

Now that we have our AMGeO environment set, let's generate some AMGeO maps! 

![AMGeO Electric Potential Map](./static/AMGeOElectricPotentialMap.png)

This is an example of an image representation of electric potential data generated by AMGeO. What this looks like in terms of raw data is as follows:

$$\begin{pmatrix}
\Phi_{1, 1} & \Phi_{1, 2} & \cdots & \Phi_{1, 37} \\
\Phi_{2, 1} & \Phi_{2, 2} & \cdots & \Phi_{2, 37} \\
\vdots & \vdots & \ddots & \vdots \\
\Phi_{24, 1} & \Phi_{24, 2} & \cdots & \Phi_{24, 37} \\
\end{pmatrix}$$

Where each column maps to a longitude in a given hemisphere, and each row maps to a latitude in a given hemisphere.

AMGeO can generate maps given many configurable options (see our documentation [here](https://amgeo.colorado.edu/protected/documentation/api.html)), but in most cases users will use default options provided by AMGeO, or some regimented subset of the given options.

AMGeO's new API has developed the concept of ```controllers``` to let users decide which out-of-the-box settings of AMGeO they would like to use. 

To get a ```controller```, run ```get_controller``` on the AMGeO API class.

```python
# Returns an AMGeO Controller
api = AMGeOApi()
api.get_controller(type='default')
>>> amgeo_controller

# Example
controller = api.get_controller() # returns the default controller by default
controller
>>> Default AMGeO Controller
```

In [ ]:
controller = api.get_controller()
controller

Currently, AMGeO only supports default settings with the AMGeO API, the only controller type supported is ```'Default'```

With our new controller, we can begin generating AMGeO Maps. Previously, to do this required using our command line interface to generate an AMGeO map. This was limited to a whole day of AMGeO or a single datetime, but now has been expanded to other high-level jobs. When generated, these maps will be stored in the grid format as numpy arrays in an hdf5 file within the AMGeO output directory.

To generate an AMGeO map, you will use the ```generate``` method on a ```controller```

```python
# generates AMGeO Data for the given arguments to the AMGeO output directory
controller.generate(date_args, hemisphere)
```

The ```date_args``` that the AMGeO API accepts are in the form of [python date/datetimes](https://docs.python.org/3/library/datetime.html)

The new ```generate``` method allows for multiple different ways of generating maps for a given task and range of dates

- <mark>Option 1: A datetime</mark>
```python
# generate data for a single datetime
dt = datetime.datetime(2013, 5, 6, 12, 30, 0) # 5/6/2013 @ 12:30:00
controller.generate(dt, 'N') # generates the map for the given datetime on the northern hemishphere
```

- <mark>Option 2: A list of datetimes</mark>
```python
# generate data for multiple datetimes
'''
NOTE: currently, this only supports a list of datetimes in the same date
this will be expanded in the future
'''
dts = [
    datetime.datetime(2013, 5, 6, 12, 30, 0), # 5/6/2013 @ 12:30:00
    datetime.datetime(2013, 5, 6, 13, 30, 0), # 5/6/2013 @ 13:30:00
    datetime.datetime(2013, 5, 6, 1, 30, 0), # 5/6/2013 @ 01:30:00
]
controller.generate(dts, 'N') # generates the maps for each datetime on the northern hemisphere
```

- <mark>Option 3: A date</mark>
```python
# generate data for a whole day, which will create maps for every 2 min 30 sec interval in a given day
d = datetime.date(2014, 1, 1) # 1/1/2014
controller.generate(d, 'S') # generates the maps for an entire day on the southern hemisphere
```

- <mark>Option 4: A range of dates</mark>
```python
# generate data for each day in a given range
date_range = (
    datetime.date(2014, 1, 1), 
    datetime.date(2014, 1, 5)
) # tuple of range from 1/1/2014 -> 1/5/2014
controller.generate(date_range, 'N')
```

Let's run some examples for the purposes of this notebook demo

In [ ]:
# First, import pythons datetime module
# This is how you will pass arguments to the AMGeO Controller to generate data
from datetime import datetime, date

In [ ]:
''' 
Example 1
Option 1: A single datetime
'''
dt = datetime(2014, 5, 6, 12, 30, 0)
hemisphere = 'N'
controller.generate(dt, hemisphere)

In [ ]:
'''
Example 2
Option 2: A list of datetimes
'''
dts = []
# do 15 min intervals for A
for i in range(0, 60, 15):
    dt = datetime(2015, 6, 25, 13, i, 0)
    print('Datetime: %s' % dt)
    dts.append(dt)
print()
controller.generate(dts, hemisphere)

In this demo notebook, these code cells run instantly because we have already generated the AMGeO maps. 

**In practice, AMGeO will take a variable amount of time depending on your system specs. Expect non-trivial times on large jobs such as a whole day**

Now that we have some assimilative maps generated, let's look into how we can load these maps into Xarray objects for effecient data interaction/operations

### Loading AMGeO Maps

With the data we just generated, let's see what days we have available to load from. The ```controller``` class provides the ```browse``` method to do this.

```python
# returns the list of all days for which AMGeO has data available in its output directory
controller.browse()
>>> [] # list of days that have maps generated 

# Example
controller.browse()
>>> ['20110101N', '20120102S']
```

In [ ]:
controller.browse()

One of the key development tasks that the API achieves is giving researchers/developers the tools to interact with AMGeO data in an intuitive way. 

[Xarray](http://xarray.pydata.org/en/stable/index.html), a new python pacakge designed to help bundle scientific data into python, is used by AMGeO to accopmlish this. 

Once you have used a controller to generate these maps, controllers also allow for you to load the data into an [Xarray Dataset](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.html)

This image is a visual aid to understand how AMGeO data is represented/stored in a Xarray Dataset

![AMGeO Xarray Dataset](./static/AMGeOXarrayDataset.png)

Now that we have some maps available to use, let's try loading one day into an Dataset. To do this, use the ```load``` method on the ```controller``` class

```python
# load data available for a given day/hemisphere stored in AMGeO's output directory
controller.load('YYYYMMDDH') 
>>> Xarray DataSet

# Example
first_day = controller.browse()[0] # grab first element in days available
print(first_day)
>>> '20190501N'
ds = controller.load(first_day) # store dataset
```

In [ ]:
# grabs the first day from our list of days that we have maps for
days = controller.browse()
day = days[0]

# load the days data into an Xarray Dataset
ds = controller.load(day)

ds

With a live AMGeO Dataset, let's learn more about Xarray and how it can aid your research goals

### Interacting with AMGeO Maps using Xarray


#### Xarray Data Variables

To first understand how Xarray Datasets work, the first step is to understand data variables. 
If the Dataset was analagous to a python dictionary, Data Variables would be this dictionary's keys. 
By accessing one, you get a [Xarray DataArray](http://xarray.pydata.org/en/stable/generated/xarray.DataArray.html) of the specific type of data you are interested in. For example, to get the electric potential data, you would access the ```epot``` key on the Dataset

In [ ]:
ds['epot']

Each of these data variables maps to a specific type of data from an AMGeO Map, such as Electric Potential, Hall Conductance, Joule Heating, etc. 

#### Xarray Dimensions

Since we are dealing with multi-dimensional data, it can sometimes be helpful to understand what dimensions you are currently working with at any given time. Xarray makes this very easy to access with the ```dims``` attribute on a DataArray

In [ ]:
halls = ds['cond_hall'] # get the hall conductance data from the dataset
halls.dims

From the above example, we can see that once we have the hall conductance data, our first dimension on the array is time (which datetime you are interested in), followed by the lat/lon grid of hall conductance observations

#### Xarray Coordinates

Xarray Coordinates allow for even further analysis of the dimensions, with information as to what the specific coordinate(s) are at a given data point. 

In [ ]:
halls.coords

From the above, we can seee that our dims ```time```, ```lat``` and ```lon``` are datatypes ```datetime64```, ```float64``` and ```float64``` respectively. We can access a specific coordinate we are interested in.

In [ ]:
# grab the first time slice in the hall conductance array
hall = halls[0]

# get the time of the specific data point in question
hall.time

#### Xarray Metadata

One of the other benefits of using Xarray is its use of metadata. AMGeO uses this capability currently and seeks to expand it even more in the future. 

To access the metadata, use Xarray's ```.attrs``` attribute

Metadata lives on all aspects of the AMGeO DataSet

From the Dataset itself...

In [ ]:
ds.attrs

to the DataArrays inside of the DataSet

In [ ]:
epots = ds['epot']
epots.attrs

These aim to help provide relevant information to the user within the notebook itself without having to go dig it up on your own

#### Compatability with Numpy

Xarray allows for simple one to one mapping operations from an Xarray DataArray to a Numpy Array using the ```.values``` attribute

In [ ]:
epots = ds['epot']
numpy_arr = epots.values 
type(numpy_arr)

#### Compatability with Pandas

All this multi-dimensial stuff not your forte? More used to pandas DataFrames? No Problem! Xarray also has a simple method to transform a DataSet to a DataFrame

While possible, this is not recommended as it can be even more taxing to move away from the multi-dimensional structure of the Dataset. BUT, it is possible

In [ ]:
ds.to_dataframe()

# Thanks! 

For ideas, comments, or overall impressions, please post them [here!](https://padlet.com/matsuot/un3cbo3q9p9yz3b)